In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
# Read pickle file of preprocessed images

pickle_file = 'SVHN_single.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    X_train = save['train_features']
    y_train = save['train_labels']
    X_valid = save['valid_features']
    y_valid = save['valid_labels']
    X_test = save['test_features']
    y_test = save['test_labels']
    del save  # Hint to help gc free up memory

print('Training set dimensions: ', X_train.shape, y_train.shape)
print('Validation set dimensions: ', X_valid.shape, y_valid.shape)
print('Testing set dimensions: ', X_test.shape, y_test.shape)

Training set dimensions:  (71257, 32, 32) (71257,)
Validation set dimensions:  (6000, 32, 32) (6000,)
Testing set dimensions:  (26032, 32, 32) (26032,)


In [3]:
# Accuracy calculations
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [4]:
# Reformatting the features and labels to prepare for model learning

img_size = 32
num_labels = 10
num_channels = 1

def reformat(X, y):
    X = X.reshape((-1, img_size, img_size, num_channels)).astype(np.float32)
    y =(np.arange(num_labels) == y[:,None]).astype(np.float32) 
    return X, y

X_train, y_train = reformat(X_train, y_train)
X_valid, y_valid = reformat(X_valid, y_valid)
X_test, y_test = reformat(X_test, y_test)

print("Training set dimensions:", X_train.shape, y_train.shape)
print("Validation set dimensions:", X_valid.shape, y_valid.shape)
print("Testing set dimensions:", X_test.shape, y_test.shape)

Training set dimensions: (71257, 32, 32, 1) (71257, 10)
Validation set dimensions: (6000, 32, 32, 1) (6000, 10)
Testing set dimensions: (26032, 32, 32, 1) (26032, 10)


In [5]:
# Convolutional Neural Network Architecture

batch_size = 128
patch_size = 5
depth_1 = 8
depth_2 = 16
depth_3 = 32
num_hidden_1 = 64
num_hidden_2 = 32

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, img_size, img_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(X_valid)
    tf_test_dataset = tf.constant(X_test)

    # Helper functions for convolution and pooling
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    
    def conv2d_s2(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='SAME')

    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], 
                              strides=[1, 2, 2, 1], padding='SAME')
    
    # Variables.
    W_conv_1 = tf.Variable(tf.truncated_normal([5, 5, 1, 8], stddev=0.1))
    b_conv_1 = tf.Variable(tf.zeros([8]))
    W_conv_2 = tf.Variable(tf.truncated_normal([5, 5, 8, 16], stddev=0.1))
    b_conv_2 = tf.Variable(tf.constant(1.0, shape=[16]))
    W_conv_3 = tf.Variable(tf.truncated_normal([5, 5, 16, 32], stddev=0.1))
    b_conv_3 = tf.Variable(tf.constant(1.0, shape=[32]))
    W_conv_4 = tf.Variable(tf.truncated_normal([3, 3, 32, 64], stddev=0.1))
    b_conv_4 = tf.Variable(tf.constant(1.0, shape=[64]))
    
    W_fc_1 = tf.Variable(tf.truncated_normal([256, 128], stddev=0.1))
    b_fc_1 = tf.Variable(tf.constant(1.0, shape=[128]))
    W_fc_2 = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1))
    b_fc_2 = tf.Variable(tf.constant(1.0, shape=[10]))

    # Model.
    def model(data, keep_prob):
        conv = conv2d(data, W_conv_1)
        hidden = tf.nn.relu(conv + b_conv_1)
        sub = max_pool_2x2(hidden)
        conv = conv2d(sub, W_conv_2)
        hidden = tf.nn.relu(conv + b_conv_2)
        sub = max_pool_2x2(hidden)
        conv = conv2d(sub, W_conv_3)
        hidden = tf.nn.relu(conv + b_conv_3)
        sub = max_pool_2x2(hidden)
        conv = conv2d(sub, W_conv_4)
        hidden = tf.nn.relu(conv + b_conv_4)
        sub = max_pool_2x2(hidden)
        drop = tf.nn.dropout(sub, keep_prob)
    
        shape = drop.get_shape().as_list()
        print(shape)
        reshape = tf.reshape(drop, [shape[0], shape[1] * shape[2] * shape[3]])
        drop = tf.nn.relu(tf.matmul(reshape, W_fc_1) + b_fc_1)
        return tf.matmul(drop, W_fc_2) + b_fc_2
    
    
    
    # Training computation.
    keep_prob = 0.5
    logits = model(tf_train_dataset, keep_prob)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    global_step = tf.Variable(0)
    starter_learning_rate = 0.001
    # Decay factor of 0.95 after every 1000 steps.
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.95)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0)) # Turn off Dropout (keep_prob = 1.0)
    test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0)) # Turn off Dropout (keep_prob = 1.0)

[128, 2, 2, 64]
[6000, 2, 2, 64]
[26032, 2, 2, 64]


In [6]:
num_steps = 30001
saver = tf.train.Saver()

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    from time import time
    t0 = time()
    
    for step in range(num_steps):
        # Split training data according to batch size.
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        batch_data = X_train[offset:(offset + batch_size), :, :, :]
        batch_labels = y_train[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # Run the model
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), y_valid))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), y_test))
    print("Total run time:", round(((time()-t0)/60), 1), "min")
    # Saves model for later use.
    save_path = saver.save(session, 'CNN_model_single.ckpt')
    print("Model saved in file: %s" % save_path)

Initialized
Minibatch loss at step 0: 165.010071
Minibatch accuracy: 9.4%
Validation accuracy: 13.6%
Minibatch loss at step 1000: 1.278368
Minibatch accuracy: 56.2%
Validation accuracy: 70.4%
Minibatch loss at step 2000: 0.830801
Minibatch accuracy: 71.1%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 0.858654
Minibatch accuracy: 69.5%
Validation accuracy: 84.7%
Minibatch loss at step 4000: 0.581120
Minibatch accuracy: 83.6%
Validation accuracy: 86.7%
Minibatch loss at step 5000: 0.796582
Minibatch accuracy: 76.6%
Validation accuracy: 88.5%
Minibatch loss at step 6000: 0.466583
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 7000: 0.448171
Minibatch accuracy: 85.9%
Validation accuracy: 90.1%
Minibatch loss at step 8000: 0.395155
Minibatch accuracy: 87.5%
Validation accuracy: 90.6%
Minibatch loss at step 9000: 0.454451
Minibatch accuracy: 86.7%
Validation accuracy: 90.9%
Minibatch loss at step 10000: 0.433170
Minibatch accuracy: 86.7%
Validation accu